In [2]:
from functools import partial
from typing import Literal
from dataclasses import dataclass

import pandas as pd
import redis

In [3]:
r = redis.Redis(db=4)

keys = r.keys()

In [11]:
keys[0].decode("utf-8")

'تا نشانده\u200cام در دل ساق سرو و سیمینتچیده\u200cام به هر دستی میوه\u200cهای سیمین را'

In [7]:
r.hgetall(keys[0])

{b'poet': b'\xd9\x81\xd8\xb1\xd9\x88\xd8\xba\xdb\x8c \xd8\xa8\xd8\xb3\xd8\xb7\xd8\xa7\xd9\x85\xdb\x8c',
 b'url': b'https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh32/',
 b'm1': b'\xd8\xaa\xd8\xa7 \xd9\x86\xd8\xb4\xd8\xa7\xd9\x86\xd8\xaf\xd9\x87\xe2\x80\x8c\xd8\xa7\xd9\x85 \xd8\xaf\xd8\xb1 \xd8\xaf\xd9\x84 \xd8\xb3\xd8\xa7\xd9\x82 \xd8\xb3\xd8\xb1\xd9\x88 \xd9\x88 \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd9\x86\xd8\xaa',
 b'm2': b'\xda\x86\xdb\x8c\xd8\xaf\xd9\x87\xe2\x80\x8c\xd8\xa7\xd9\x85 \xd8\xa8\xd9\x87 \xd9\x87\xd8\xb1 \xd8\xaf\xd8\xb3\xd8\xaa\xdb\x8c \xd9\x85\xdb\x8c\xd9\x88\xd9\x87\xe2\x80\x8c\xd9\x87\xd8\xa7\xdb\x8c \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd9\x86 \xd8\xb1\xd8\xa7'}

### Here we write some structures and functions for handling Beyt objects

In [12]:
def load_beyt(data: dict[bytes, bytes]) -> dict[str, str]:
    return {k.decode("utf-8"): v.decode("utf-8") 
            for k, v in data.items()
        }


@dataclass
class Beyt:
    m1: str
    m2: str
    poet: str
    url: str

    @property
    def key(self):
        return self.m1 + self.m2

In [13]:
Beyt(**load_beyt(r.hgetall(keys[0])))

Beyt(m1='تا نشانده\u200cام در دل ساق سرو و سیمینت', m2='چیده\u200cام به هر دستی میوه\u200cهای سیمین را', poet='فروغی بسطامی', url='https://ganjoor.net/forooghi/divan-forooghi/ghazalf/sh32/')

In [14]:
db = []

for key in keys:
    db.append(Beyt(**load_beyt(r.hgetall(key))))

## Start Create DataFrame

In [16]:
df = pd.DataFrame(data=db)
df

,m1,m2,poet,url
0,تا نشانده‌ام در دل ساق سرو و سیمینت,چیده‌ام به هر دستی میوه‌های سیمین را,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...
1,الا ای رو ترش کرده که تا نبود مرا مدخل,نبشته گرد روی خود صلا نعم الادام الخل,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
2,تر کنم از آب چشم روی چونان خشک را,چون جگری بیش نیست سوخته بر خوان دل,سیف فرغانی,https://ganjoor.net/seyf/divan-seyf/ghazalha/s...
3,ای دل غم زده از عجز تو معلومم شد,که بر تیغ محبت سپر انداخته‌ای,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...
4,از می و مطرب مکن، مدعیا منع من,تا غزلی‌تر بود، قول تو خواهم شنید,سلمان ساوجی,https://ganjoor.net/salman/divanss/ghazalss/sh...
...,...,...,...,...
74439,صد نامه فرستادم صد راه نشان دادم,یا راه نمی‌دانی یا نامه نمی‌خوانی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...
74440,آن که دل من چو گوی در خم چوگان اوست,موقف آزادگان بر سر میدان اوست,سعدی,https://ganjoor.net/saadi/divan/ghazals/sh95/
74441,خون در دل و ریشهٔ تنم سوخت چنان,کز دیده بجای اشک خاکستر ریخت,ابوسعید ابوالخیر,https://ganjoor.net/abusaeed/robaee-aa/sh44/
74442,گردون برای خیمه خورشید فلکه‌ات,از کوه و ابر ساخته نازیر و سایه‌بان,حافظ,https://ganjoor.net/hafez/ghaside/sh1/


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74444 entries, 0 to 74443
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   m1      74444 non-null  object
 1   m2      74444 non-null  object
 2   poet    74444 non-null  object
 3   url     74444 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [18]:
# Find Counts of Poems URL

df.url.value_counts()

## IF YOU COLLECTED ENORMOUS DATA OF A POEM URL THEN YOU CAUGHT ALL BEYTS(COUPLET)  :)))

url
https://ganjoor.net/hafez/saghiname/                      58
https://ganjoor.net/moulavi/shams/ghazalsh/sh1940/        47
https://ganjoor.net/hafez/ghaside/sh3/                    43
https://ganjoor.net/hafez/ghaside/sh2/                    40
https://ganjoor.net/hafez/ghaside/sh1/                    40
                                                          ..
https://ganjoor.net/ouhadi/divano/robaeeo/sh162/           1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh409/         1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh329/         1
https://ganjoor.net/saeb/divan-saeb/takbeit/sh41/          1
https://ganjoor.net/khosro/gozide/ghazal-khosro/sh317/     1
Name: count, Length: 14897, dtype: int64

In [19]:
# Get All Hafez Poems from db

df[df.poet == "حافظ"] # == df[df.poet.str.contains("حافظ")]

,m1,m2,poet,url
78,در چین زلفش ای دل مسکین چگونه‌ای,کشفته گفت باد صبا شرح حال تو,حافظ,https://ganjoor.net/hafez/ghazal/sh408/
84,سحرگهم چه خوش آمد که بلبلی گلبانگ,به غنچه می‌زد و می‌گفت در سخنرانی,حافظ,https://ganjoor.net/hafez/ghaside/sh2/
85,شود چون بید لرزان سرو آزاد,اگر بیند قد دلجوی فرخ,حافظ,https://ganjoor.net/hafez/ghazal/sh99/
95,بر طرف گلشنم گذر افتاد وقت صبح,آن دم که کار مرغ سحر آه و ناله بود,حافظ,https://ganjoor.net/hafez/ghazal/sh214/
138,مستی به آب یک دو عنب وضع بنده نیست,من سالخورده پیر خرابات پرورم,حافظ,https://ganjoor.net/hafez/ghazal/sh329/
...,...,...,...,...
74371,سوختم در چاه صبر از بهر آن شمع چگل,شاه ترکان فارغ است از حال ما کو رستمی,حافظ,https://ganjoor.net/hafez/ghazal/sh470/
74372,بخت حافظ گر از این گونه مدد خواهد کرد,زلف معشوقه به دست دگران خواهد بود,حافظ,https://ganjoor.net/hafez/ghazal/sh205/
74416,هم عفاالله صبا کز تو پیامی می‌داد,ور نه در کس نرسیدیم که از کوی تو بود,حافظ,https://ganjoor.net/hafez/ghazal/sh210/
74428,به خلدم دعوت ای زاهد مفرما,که این سیب زنخ زان بوستان به,حافظ,https://ganjoor.net/hafez/ghazal/sh419/


#### cleaning m1, m1

In [20]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,m1,m2,poet,url
1659,...,زنده مانم چو در آمدز در معشوقی,محتشم کاشانی,https://ganjoor.net/mohtasham/divan-moh/ghazal-moh/sh574/
62795,***,ای فخر من سلطان من سلطان من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1785/


In [21]:
df[df.url == "https://ganjoor.net/moulavi/shams/ghazalsh/sh1785/"]

,m1,m2,poet,url
954,فرمان ده و خاقان من خاقان من,چون سوی من میلی کنی میلی کنی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
14722,روز و شبم مونس تویی مونس تویی,دام مرا خوش آهویی خوش آهویی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
14969,در خانه‌ام چون روزنی چون روزنی,تیر بلا چون دررسد چون دررسد,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
16163,صبر مرا برهم زدی برهم زدی,عقل مرا رهزن شدی رهزن شدی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
25556,چون سایه‌ها در چاشتگه,فتح و ظفر پیشت دود پیشت دود,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
33951,تویی تویی گلزار من گلزار من,بگو بگو اسرار من اسرار من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
39917,بیا بیا دلدار من دلدار من,درآ درآ در کار من در کار من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
53579,هر جا روم با من روی با من روی,هر منزلی محرم شوی محرم شوی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...
62795,***,ای فخر من سلطان من سلطان من,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...


In [24]:
df.drop(df[df.url == "https://ganjoor.net/moulavi/shams/ghazalsh/sh1785/"].index, inplace=True)

In [25]:
# df.loc[62795, "m1"] = "ای فخر من سلطان من سلطان من"
# df.loc[62795, "m2"] = "فرمان ده و خاقان من خاقان من"

df.drop(1659, inplace=True)

df.reset_index(inplace=True)
##  Full Handy & IRREVERSIBLE!
# Remove this part!

In [26]:
df[(df.m1.str.len() < 10) | (df.m2.str.len() < 10)].style

,index,m1,m2,poet,url


In [27]:
ocr_err = df[df.m1 == "ای دل دو سه شام تا سحرگاه مخسب"]
print(ocr_err[["m1", "m2"]])
if not ocr_err.empty:
    idx = ocr_err.index
    df.loc[idx, "m2"] = "در فرقت آفتاب چون ماه مخسب"
    print("FIXED")
    print(df.loc[idx, ["m1", "m2"]])


                                   m1                          m2
27871  ای دل دو سه شام تا سحرگاه مخسب  در فرقت آفتاب چون ماه مخسپ
FIXED
                                   m1                          m2
27871  ای دل دو سه شام تا سحرگاه مخسب  در فرقت آفتاب چون ماه مخسب


#### creating start, end word

In [29]:
# Start , Ends Cleaning


def clean_and_select(string: str, mode: Literal["start", "end"] = "start"):
    clean_dict = {
        "آ": "ا",
        "ء": "ا",
        "ة": "ت",
    }
    striped = string.strip(" \n\t.؟!،”:()«»")

    match mode:
        case "start":
            res = clean_dict.get(striped[0], striped[0])
        case "end":
            res = clean_dict.get(striped[-1], striped[-1])
        case _:
            raise ValueError(f'mode: Literal["start", "end"] (got {mode})')

    return res

In [30]:
df["start"] = df.m1.apply(partial(clean_and_select, mode="start"))
df["end"] = df.m2.apply(partial(clean_and_select, mode="end"))

df

,index,m1,m2,poet,url,start,end
0,0,تا نشانده‌ام در دل ساق سرو و سیمینت,چیده‌ام به هر دستی میوه‌های سیمین را,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...,ت,ا
1,1,الا ای رو ترش کرده که تا نبود مرا مدخل,نبشته گرد روی خود صلا نعم الادام الخل,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh1...,ا,ل
2,2,تر کنم از آب چشم روی چونان خشک را,چون جگری بیش نیست سوخته بر خوان دل,سیف فرغانی,https://ganjoor.net/seyf/divan-seyf/ghazalha/s...,ت,ل
3,3,ای دل غم زده از عجز تو معلومم شد,که بر تیغ محبت سپر انداخته‌ای,فروغی بسطامی,https://ganjoor.net/forooghi/divan-forooghi/gh...,ا,ی
4,4,از می و مطرب مکن، مدعیا منع من,تا غزلی‌تر بود، قول تو خواهم شنید,سلمان ساوجی,https://ganjoor.net/salman/divanss/ghazalss/sh...,ا,د
...,...,...,...,...,...,...,...
74429,74439,صد نامه فرستادم صد راه نشان دادم,یا راه نمی‌دانی یا نامه نمی‌خوانی,مولوی,https://ganjoor.net/moulavi/shams/ghazalsh/sh2...,ص,ی
74430,74440,آن که دل من چو گوی در خم چوگان اوست,موقف آزادگان بر سر میدان اوست,سعدی,https://ganjoor.net/saadi/divan/ghazals/sh95/,ا,ت
74431,74441,خون در دل و ریشهٔ تنم سوخت چنان,کز دیده بجای اشک خاکستر ریخت,ابوسعید ابوالخیر,https://ganjoor.net/abusaeed/robaee-aa/sh44/,خ,ت
74432,74442,گردون برای خیمه خورشید فلکه‌ات,از کوه و ابر ساخته نازیر و سایه‌بان,حافظ,https://ganjoor.net/hafez/ghaside/sh1/,گ,ن


In [31]:
# list of words that starts the m1
starts = df.start.value_counts()
starts, len(df.start.value_counts())

(start
 ا    10581
 ب     8729
 د     6443
 م     6020
 گ     4693
 چ     4568
 ه     3492
 ت     3330
 ز     2849
 ن     2736
 خ     2591
 س     2346
 ک     1980
 ش     1821
 ع     1623
 ر     1583
 ج     1396
 ی     1156
 و     1133
 پ     1078
 ح      822
 ص      781
 ف      701
 غ      527
 ق      520
 ل      471
 ط      315
 ذ       78
 ظ       35
 ض       21
 ث       11
 ژ        4
 Name: count, dtype: int64,
 32)

In [32]:
# list of words that ends the m2
ends = df.end.value_counts()
ends, len(df.end.value_counts())

(end
 د    15696
 ی    13511
 م    10221
 ت     9988
 ن     5836
 ا     5008
 ر     2542
 ش     2372
 ه     2285
 و     2257
 ل     1023
 ز      988
 ب      863
 س      388
 ق      285
 ک      268
 گ      140
 غ      131
 ف       97
 ح       94
 ع       94
 ج       80
 ظ       48
 خ       40
 ث       39
 چ       34
 ص       32
 ض       26
 ذ       24
 ط       24
 Name: count, dtype: int64,
 30)

In [33]:
end_scale = ends.sum() / ends
end_scale

end
د       4.742227
ی       5.509141
م       7.282458
ت       7.452343
ن      12.754284
ا      14.863019
ر      29.281668
ش      31.380270
ه      32.575055
و      32.979176
ل      72.760508
ز      75.338057
ب      86.250290
س     191.840206
ق     261.171930
ک     277.738806
گ     531.671429
غ     568.198473
ف     767.360825
ح     791.851064
ع     791.851064
ج     930.425000
ظ    1550.708333
خ    1860.850000
ث    1908.564103
چ    2189.235294
ص    2326.062500
ض    2862.846154
ذ    3101.416667
ط    3101.416667
Name: count, dtype: float64

In [34]:
# What word not exist in ends of m2?

set(df.start.unique()) - set(df.end.unique())

{'پ', 'ژ'}

In [36]:
#### variety of poetry

dfg = df.groupby("poet")

df_poet = dfg.agg("count").sort_values("m1")

df_poet

,index,m1,m2,url,start,end
poet,,,,,,
خیام,354,354,354,354,354,354
هاتف اصفهانی,559,559,559,559,559,559
رهی معیری,758,758,758,758,758,758
باباطاهر,760,760,760,760,760,760
عبید زاکانی,827,827,827,827,827,827
سیف فرغانی,1336,1336,1336,1336,1336,1336
ابوسعید ابوالخیر,1439,1439,1439,1439,1439,1439
شهریار,1508,1508,1508,1508,1508,1508
امیرخسرو دهلوی,1671,1671,1671,1671,1671,1671


In [37]:
df_poet.index

Index(['خیام', 'هاتف اصفهانی', 'رهی معیری', 'باباطاهر', 'عبید زاکانی',
       'سیف فرغانی', 'ابوسعید ابوالخیر', 'شهریار', 'امیرخسرو دهلوی', 'صائب',
       'عراقی', 'سلمان ساوجی', 'حافظ', 'اوحدی', 'خواجوی کرمانی',
       'محتشم کاشانی', 'فروغی بسطامی', 'سعدی', 'مولوی'],
      dtype='object', name='poet')

In [38]:
#### starts and ends in poetry of poet

sample_poet = df_poet.index[0]
print(sample_poet)
dfg.get_group(sample_poet).start.unique() , len(dfg.get_group(sample_poet).start.unique())

خیام


(array(['م', 'چ', 'ب', 'ا', 'ک', 'و', 'د', 'گ', 'ت', 'پ', 'ق', 'خ', 'ز',
        'ن', 'ر', 'ع', 'س', 'ج', 'ی', 'ل', 'ه', 'ح', 'ش', 'ف', 'ص', 'ض'],
       dtype=object),
 26)

In [88]:
# and ends
dfg.get_group(sample_poet).end.unique() , len(dfg.get_group(sample_poet).end.unique())

(array(['ن', 'ر', 'ه', 'د', 'م', 'ب', 'ی', 'و', 'ت', 'ل', 'ز', 'س', 'گ',
        'ا', 'خ', 'ش'], dtype=object),
 16)

In [42]:
# create counts of words in starts of each poets

starts_dict = {poet: len(dfg.get_group(poet).start.unique()) for poet in df_poet.index}
ends_dict = {poet: len(dfg.get_group(poet).end.unique()) for poet in df_poet.index}

starts_dict
ends_dict

{'خیام': 16,
 'هاتف اصفهانی': 17,
 'رهی معیری': 15,
 'باباطاهر': 17,
 'عبید زاکانی': 13,
 'سیف فرغانی': 17,
 'ابوسعید ابوالخیر': 22,
 'شهریار': 16,
 'امیرخسرو دهلوی': 21,
 'صائب': 20,
 'عراقی': 18,
 'سلمان ساوجی': 17,
 'حافظ': 23,
 'اوحدی': 19,
 'خواجوی کرمانی': 23,
 'محتشم کاشانی': 29,
 'فروغی بسطامی': 15,
 'سعدی': 18,
 'مولوی': 25}

In [43]:
df_count_alphabet_start = pd.DataFrame(
    data=starts_dict.values(),
    index=starts_dict.keys(),
    columns=["count_alphabet_start"],
)
df_count_alphabet_end = pd.DataFrame(
    data=ends_dict.values(),
    index=ends_dict.keys(),
    columns=["count_alphabet_end"],
)

df_count_alphabet = df_count_alphabet_start.merge(
    df_count_alphabet_end,
    left_index=True,
    right_index=True,
)

df_count = df_poet.merge(
    df_count_alphabet,
    left_index=True,
    right_index=True,
)[["start", "count_alphabet_start", "count_alphabet_end"]].rename(columns={"start": "poem_count"})

df_count

,poem_count,count_alphabet_start,count_alphabet_end
خیام,354,26,16
هاتف اصفهانی,559,26,17
رهی معیری,758,27,15
باباطاهر,760,27,17
عبید زاکانی,827,29,13
سیف فرغانی,1336,30,17
ابوسعید ابوالخیر,1439,30,22
شهریار,1508,29,16
امیرخسرو دهلوی,1671,29,21
صائب,2849,29,20


In [47]:
from functools import reduce

In [48]:
# create counts of words in starts of each poets

starts_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).start.unique())) for poet in df_poet.index}
ends_dict = {poet: reduce(lambda x, y: x+y , sorted(dfg.get_group(poet).end.unique())) for poet in df_poet.index}

starts_dict
# ends_dict

{'خیام': 'ابتجحخدرزسشصضعفقلمنهوپچکگی',
 'هاتف اصفهانی': 'ابتجحخدرزسشصعغفقلمنهوپچکگی',
 'رهی معیری': 'ابتجحخدرزسشصطعغفقلمنهوپچکگی',
 'باباطاهر': 'ابتجحخدرزسشصطعغفقلمنهوپچکگی',
 'عبید زاکانی': 'ابتجحخدذرزسشصضطعغفقلمنهوپچکگی',
 'سیف فرغانی': 'ابتجحخدذرزسشصطظعغفقلمنهوپچژکگی',
 'ابوسعید ابوالخیر': 'ابتجحخدذرزسشصضطظعغفقلمنهوپچکگی',
 'شهریار': 'ابتثجحخدذرزسشصطعغفقلمنهوپچکگی',
 'امیرخسرو دهلوی': 'ابتجحخدذرزسشصطعغفقلمنهوپچژکگی',
 'صائب': 'ابتجحخدذرزسشصضطعغفقلمنهوپچکگی',
 'عراقی': 'ابتجحخدذرزسشصطظعغفقلمنهوپچکگی',
 'سلمان ساوجی': 'ابتجحخدذرزسشصطظعغفقلمنهوپچکگی',
 'حافظ': 'ابتثجحخدذرزسشصضطظعغفقلمنهوپچکگی',
 'اوحدی': 'ابتثجحخدذرزسشصضطعغفقلمنهوپچکگی',
 'خواجوی کرمانی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوپچکگی',
 'محتشم کاشانی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوپچکگی',
 'فروغی بسطامی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوپچژکگی',
 'سعدی': 'ابتثجحخدذرزسشصضطظعغفقلمنهوپچکگی',
 'مولوی': 'ابتجحخدذرزسشصضطظعغفقلمنهوپچژکگی'}

In [94]:
def remove_common_letters(dictionary):
    common_letters = set.intersection(*map(set, dictionary.values()))

    new_dictionary = {
        key: "".join(filter(lambda letter: letter not in common_letters, value))
        for key, value in dictionary.items()
    }

    return new_dictionary


remove_common_letters(starts_dict)
# remove_common_letters(ends_dict)

{'خیام': 'ض',
 'هاتف اصفهانی': 'غ',
 'رهی معیری': 'طغ',
 'باباطاهر': 'طغ',
 'عبید زاکانی': 'ذضطغ',
 'سیف فرغانی': 'ذطظغژ',
 'ابوسعید ابوالخیر': 'ذضطظغ',
 'شهریار': 'ثذطغ',
 'امیرخسرو دهلوی': 'ذطغژ',
 'صائب': 'ذضطغ',
 'عراقی': 'ذطظغ',
 'سلمان ساوجی': 'ذطظغ',
 'حافظ': 'ثذضطظغ',
 'اوحدی': 'ثذضطغ',
 'خواجوی کرمانی': 'ثذضطظغ',
 'محتشم کاشانی': 'ثذضطظغ',
 'فروغی بسطامی': 'ثذضطظغژ',
 'سعدی': 'ثذضطظغ',
 'مولوی': 'ذضطظغژ'}

In [50]:
poet_score_ends = {}

for poet, words in ends_dict.items():
    poet_score_end = 0
    for word in words:
        poet_score_end += end_scale[word]
    poet_score_ends[poet] = poet_score_end

df_final = df_count.merge(
    pd.DataFrame(
        poet_score_ends.values(),
        index=poet_score_ends.keys(),
        columns=["poet_score_ends"],
    ),
    right_index=True,
    left_index=True,
).sort_values(by=["poet_score_ends"])

df_final

,poem_count,count_alphabet_start,count_alphabet_end,poet_score_ends
عبید زاکانی,827,29,13,911.860747
فروغی بسطامی,5662,32,15,952.079230
رهی معیری,758,27,15,1482.758364
سلمان ساوجی,3343,29,17,2226.230168
عراقی,3268,29,18,2243.789338
هاتف اصفهانی,559,26,17,2328.277059
خیام,354,26,16,2997.530129
شهریار,1508,29,16,3332.500570
صائب,2849,29,20,3827.491466
سیف فرغانی,1336,30,17,4071.739524


In [52]:
df_final["score"] = (
    df_final["count_alphabet_start"]
    # * df_final["count_alphabet_end"]
    * df_final["poet_score_ends"]
    / df_final["poem_count"]
)
df_final.sort_values("score")

,poem_count,count_alphabet_start,count_alphabet_end,poet_score_ends,score
فروغی بسطامی,5662,32,15,952.079230,5.380879
مولوی,22904,31,25,12157.537547,16.454928
سلمان ساوجی,3343,29,17,2226.230168,19.312197
عراقی,3268,29,18,2243.789338,19.911227
سعدی,7283,31,18,5067.467199,21.569612
اوحدی,4845,30,19,4656.677223,28.833915
عبید زاکانی,827,29,13,911.860747,31.975770
صائب,2849,29,20,3827.491466,38.960075
رهی معیری,758,27,15,1482.758364,52.815931
خواجوی کرمانی,5205,31,23,9110.150085,54.258339
